<a href="https://colab.research.google.com/github/medaminechafii/spaceship-titanic/blob/XGBoost-model/mynewproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [497]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [498]:
data = pd.read_csv("train.csv")
data_test = pd.read_csv("test.csv")

X = data.iloc[:,:-1]
y = data.iloc[:,-1]

In [499]:
X

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre


In [500]:
X.drop(["PassengerId","Name","Age","Cabin"],axis=1,inplace=True)

In [501]:
X.head()

,HomePlanet,CryoSleep,Destination,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
0,Europa,False,TRAPPIST-1e,False,0.0,0.0,0.0,0.0,0.0
1,Earth,False,TRAPPIST-1e,False,109.0,9.0,25.0,549.0,44.0
2,Europa,False,TRAPPIST-1e,True,43.0,3576.0,0.0,6715.0,49.0
3,Europa,False,TRAPPIST-1e,False,0.0,1283.0,371.0,3329.0,193.0
4,Earth,False,TRAPPIST-1e,False,303.0,70.0,151.0,565.0,2.0


In [502]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2,random_state = 0)

# seperating categorical and numerical categories

In [503]:
object_cols = [col for col in X.columns if X_train[col].dtype =="object"]
num_cols = [col for col in X.columns if X_train[col].dtype in ["int64","float64"]]

In [504]:
print(object_cols)

['HomePlanet', 'CryoSleep', 'Destination', 'VIP']


In [505]:
print(num_cols)

['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']


# imputation

In [506]:
from sklearn.impute import SimpleImputer

In [507]:
numerical_tranformer = SimpleImputer(strategy = "constant")
X_num_imputed = pd.DataFrame(numerical_tranformer.fit_transform(X_train[num_cols]))
X_num_imputed.columns = num_cols
X_num_imputed.index = X_train.index
X_num_imputed

,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
4278,0.0,559.0,0.0,15238.0,2799.0
5971,0.0,20.0,1.0,696.0,0.0
464,1821.0,0.0,47.0,29.0,0.0
4475,185.0,0.0,476.0,1810.0,53.0
8469,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...
4373,0.0,0.0,0.0,0.0,0.0
7891,0.0,0.0,6.0,0.0,733.0
4859,523.0,0.0,21.0,4.0,811.0
3264,0.0,0.0,0.0,0.0,0.0


In [508]:
X_train[num_cols] = X_num_imputed
X_train

,HomePlanet,CryoSleep,Destination,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
4278,Europa,False,55 Cancri e,False,0.0,559.0,0.0,15238.0,2799.0
5971,Earth,False,TRAPPIST-1e,False,0.0,20.0,1.0,696.0,0.0
464,Mars,False,TRAPPIST-1e,False,1821.0,0.0,47.0,29.0,0.0
4475,Earth,False,TRAPPIST-1e,False,185.0,0.0,476.0,1810.0,53.0
8469,Europa,True,55 Cancri e,False,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
4373,Europa,True,55 Cancri e,False,0.0,0.0,0.0,0.0,0.0
7891,Earth,False,TRAPPIST-1e,False,0.0,0.0,6.0,0.0,733.0
4859,Mars,False,TRAPPIST-1e,False,523.0,0.0,21.0,4.0,811.0
3264,Earth,False,TRAPPIST-1e,False,0.0,0.0,0.0,0.0,0.0


In [509]:
categorical_transformer = SimpleImputer(strategy = "most_frequent")
X_cat_imputed = pd.DataFrame(categorical_transformer.fit_transform(X_train[object_cols]))
X_cat_imputed.columns = object_cols
X_cat_imputed.index = X_train.index
X_train[object_cols] = X_cat_imputed
X_train

,HomePlanet,CryoSleep,Destination,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
4278,Europa,False,55 Cancri e,False,0.0,559.0,0.0,15238.0,2799.0
5971,Earth,False,TRAPPIST-1e,False,0.0,20.0,1.0,696.0,0.0
464,Mars,False,TRAPPIST-1e,False,1821.0,0.0,47.0,29.0,0.0
4475,Earth,False,TRAPPIST-1e,False,185.0,0.0,476.0,1810.0,53.0
8469,Europa,True,55 Cancri e,False,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
4373,Europa,True,55 Cancri e,False,0.0,0.0,0.0,0.0,0.0
7891,Earth,False,TRAPPIST-1e,False,0.0,0.0,6.0,0.0,733.0
4859,Mars,False,TRAPPIST-1e,False,523.0,0.0,21.0,4.0,811.0
3264,Earth,False,TRAPPIST-1e,False,0.0,0.0,0.0,0.0,0.0


In [510]:
na_columns = X_train.isnull().sum()
print(na_columns)

HomePlanet      0
CryoSleep       0
Destination     0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
dtype: int64


# One Hot Encoding

In [511]:
from sklearn.preprocessing import OneHotEncoder
OH = OneHotEncoder(handle_unknown = "ignore",sparse_output = False)
X_cat_OH = pd.DataFrame(OH.fit_transform(X_train[object_cols]))
X_cat_OH.index = X_train.index
X_train.drop(object_cols,axis = 1,inplace = True)
X_train_OH = pd.concat([X_train,X_cat_OH],axis = 1)
X_train_OH.columns = X_train_OH.columns.astype(str)
X_train_OH


,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,0,1,2,3,4,5,6,7,8,9
4278,0.0,559.0,0.0,15238.0,2799.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
5971,0.0,20.0,1.0,696.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
464,1821.0,0.0,47.0,29.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
4475,185.0,0.0,476.0,1810.0,53.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
8469,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4373,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
7891,0.0,0.0,6.0,0.0,733.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
4859,523.0,0.0,21.0,4.0,811.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
3264,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0


# encoding the dependant variable

In [512]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train_OH = pd.DataFrame(le.fit_transform(y_train))
y_train_OH.index = y_train.index
y_train_OH

,0
4278,0
5971,0
464,0
4475,0
8469,1
...,...
4373,1
7891,0
4859,0
3264,0


# preparing the test data

In [513]:
X_test_num_imputed = pd.DataFrame(numerical_tranformer.transform(X_test[num_cols]))
X_test_num_imputed.columns = num_cols
X_test_num_imputed.index = X_test.index
X_test[num_cols] = X_test_num_imputed

In [514]:
X_test_cat_imputed = pd.DataFrame(categorical_transformer.transform(X_test[object_cols]))
X_test_cat_imputed.columns = object_cols
X_test_cat_imputed.index = X_test.index
X_test[object_cols] = X_test_cat_imputed

In [515]:
X_test_cat_OH = pd.DataFrame(OH.transform(X_test[object_cols]))
X_test_cat_OH.index = X_test.index
X_test.drop(object_cols,axis = 1,inplace = True)
X_test_OH = pd.concat([X_test,X_test_cat_OH],axis = 1)
X_test_OH.columns = X_test_OH.columns.astype(str)

In [516]:
X_test_OH

,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,0,1,2,3,4,5,6,7,8,9
3601,0.0,0.0,623.0,40.0,209.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
6057,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
2797,517.0,1.0,301.0,60.0,4.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
7110,653.0,0.0,4.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
8579,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
864,0.0,760.0,234.0,2761.0,81.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
3780,7.0,61.0,110.0,708.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
6046,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
170,0.0,0.0,8.0,1072.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0


In [517]:
y_test_OH = le.transform(y_test)

In [518]:
y_test_OH

array([1, 1, 0, ..., 1, 0, 1])

# XGBoost model

In [519]:
!pip install scikit-learn==1.5.2

In [520]:
from xgboost import XGBClassifier
model = XGBClassifier(n_estimators = 1000,learning_rate = 0.05,early_stopping_rounds = 5,)
model.fit(X_train_OH,y_train_OH,eval_set = [(X_test_OH,y_test_OH)],verbose = False)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/_tags.py:354: FutureWarning: The XGBClassifier or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.


AttributeError: 'super' object has no attribute '__sklearn_tags__'

AttributeError: 'super' object has no attribute '__sklearn_tags__'

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=5,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=1000, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [521]:
!pip uninstall scikit-learn -y
!pip install scikit-learn==1.5.2

Found existing installation: scikit-learn 1.5.2
Uninstalling scikit-learn-1.5.2:
  Successfully uninstalled scikit-learn-1.5.2
  Using cached scikit_learn-1.5.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (13 kB)
Using cached scikit_learn-1.5.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.3 MB)


# Predictions and score

In [522]:
y_pred = model.predict(X_test_OH)

In [523]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test_OH, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[618 245]
 [133 743]]


0.7826336975273146